# User Recommend

- User recommend toma como parámetro un año particular, y retorna el top 3 de juegos **más** recomendados por usuarios para dicho año.

- Para evitar consultas lentas, crearemos una tabla auxiliar que contendrá únicamente **item_id, year_posted,	recommend,	sentiment_analysis**  la cual consultaremos finalmente en la api.

- Los comentarios van desde el 2010 hasta el 2016

In [11]:
import pandas as pd
import os

path_reviews = os.path.join('..','data','clear','user_reviews.csv.gz')
path_steam_games = os.path.join('..','data','clear','steam_games.csv.gz')


## Carga de data

In [12]:
## Cargamos la data necesaria para solucionar este problema.

steam_colums = ['id','app_name']
# steam_colums = ['id','app_name','release_year']
steam_games = pd.read_csv(path_steam_games, usecols = steam_colums)

## renombraremos la columna id por steam_id
steam_games.rename(columns={'id':'steam_id'}, inplace=True)


reviews_cols = ['item_id', 'sentiment_analysis', 'recommend', 'year_posted']
# reviews_cols = ['item_id', 'sentiment_analysis', 'recommend']
user_reviews = pd.read_csv(path_reviews, compression='gzip',usecols=reviews_cols,lineterminator='\n')


In [13]:

steam_games.head(4)

,app_name,steam_id
0,Lost summoner kitty,761140.0
1,Ironbound,643980.0
2,Real pool 3d - poolians,670290.0
3,弹炸人2222,767400.0


## Creación de tabla consulta

- En esta sección combinaremos **user_reviews** con **user_items**, por medio de .

- Vamos a hacer un join con steam_games y user_reviews,
dado que necesitamos los juego mas recomendados por usuarios, necesitamos los usuarios que hicieron almenos un comentario. es por esto que utilizaremos un inner join para no tener comentarios de usuarios desconocidos y usuarios que no realizaron comentarios



In [14]:
user_reviews.head(4)

,item_id,year_posted,recommend,sentiment_analysis
0,1250,2011,1,2
1,22200,2011,1,1
2,43110,2011,1,2
3,251610,2014,1,2


In [15]:
user_reviews

,item_id,year_posted,recommend,sentiment_analysis
0,1250,2011,1,2
1,22200,2011,1,1
2,43110,2011,1,2
3,251610,2014,1,2
4,227300,2013,1,2
...,...,...,...,...
49181,730,2015,1,1
49182,253980,2015,1,2
49183,730,2015,1,0
49184,730,2015,1,2


In [16]:
## Con fecha de steam
# user_reviews[(user_reviews['item_id'] == 33930) & (user_reviews['recommend']==1)& ((steam_games['release_year'] == 2010) )]

In [17]:
user_reviews[(user_reviews['item_id'] == 33930) & (user_reviews['recommend']==1)& ((user_reviews['year_posted'] == 2010) )]

,item_id,year_posted,recommend,sentiment_analysis


In [18]:
mask = (user_reviews['recommend'] == 1 ) & (user_reviews['sentiment_analysis'].isin([2,1])) 
user_reviews_recommend = user_reviews[mask]

In [19]:
steam_reviews = steam_games.merge(user_reviews_recommend,
                              left_on='steam_id', 
                              right_on= 'item_id',
                              how='inner').drop(columns=['steam_id','item_id'])

In [20]:
# con steams
# steam_reviews[(steam_reviews['app_name'] == 'Arma 2: operation arrowhead') & (steam_reviews['release_year']==2010)]

In [21]:
tabla_endpoint_3 = steam_reviews[['year_posted','app_name','recommend']].groupby(['year_posted','app_name']).sum()

In [22]:
tabla_endpoint_3

recommend
year_posted app_name                            
2010        Alien swarm                        4
            Amnesia: the dark descent          1
            Arcania                            1
            Audiosurf                          1
            Chime                              1
...                                          ...
2015        Zombi                              1
            Zombie army trilogy                5
            Zombie exodus                      1
            Zoombinis                          3
            Астролорды: облако оорта           1

[3653 rows x 1 columns]

## Carga de Data

In [23]:
path_endpoint_3 = os.path.join('..','data','clear','03_users_recommend.csv.gz')

tabla_endpoint_3.to_csv(path_endpoint_3,compression='gzip')

In [53]:
year = 2015

In [54]:
df_filtrado = steam_reviews[(steam_reviews['year_posted'] == year) &(steam_reviews['recommend'] == 1) & (steam_reviews['sentiment_analysis'].isin([1, 2]))]
df_filtrado[df_filtrado['app_name']=="Sid meier's civilization® v"]

,app_name,year_posted,recommend,sentiment_analysis
2687,Sid meier's civilization® v,2015,1,2
2690,Sid meier's civilization® v,2015,1,1
2693,Sid meier's civilization® v,2015,1,1
2695,Sid meier's civilization® v,2015,1,2
2706,Sid meier's civilization® v,2015,1,1
...,...,...,...,...
2924,Sid meier's civilization® v,2015,1,1
2932,Sid meier's civilization® v,2015,1,2
2938,Sid meier's civilization® v,2015,1,2
2947,Sid meier's civilization® v,2015,1,2


## Lectura de tabla consulta

In [25]:
path_endpoint_3 = os.path.join('..','data','clear','03_users_recommend.csv.gz')
table_3 = pd.read_csv(path_endpoint_3)

In [35]:
table_3

,year_posted,app_name,recommend
0,2010,Alien swarm,4
1,2010,Amnesia: the dark descent,1
2,2010,Arcania,1
3,2010,Audiosurf,1
4,2010,Chime,1
...,...,...,...
3648,2015,Zombi,1
3649,2015,Zombie army trilogy,5
3650,2015,Zombie exodus,1
3651,2015,Zoombinis,3


In [27]:
print(table_3.info(memory_usage='deep'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3653 entries, 0 to 3652
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   year_posted  3653 non-null   int64 
 1   app_name     3653 non-null   object
 2   recommend    3653 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 350.8 KB
None


In [36]:
year = 2015


In [29]:
table_3[['app_name','recommend']][table_3['year_posted']==year].groupby('app_name').sum()

,recommend
app_name,
//n.p.p.d. rush//- the milk of ultraviolet,1
"10,000,000",1
100% orange juice,4
12 labours of hercules,1
12 labours of hercules ii: the cretan bull,1
...,...
Zombi,1
Zombie army trilogy,5
Zombie exodus,1


In [57]:
table_3[['app_name','recommend']][table_3['year_posted']==year].groupby('app_name').sum().sort_values('recommend',ascending=False).head(10)

,recommend
app_name,
Counter-strike: global offensive,1401
Team fortress 2,610
Garry's mod,336
Dota 2,289
Payday 2,264
Unturned,255
Grand theft auto v,193
Terraria,189
Rust,148


In [30]:
top_3 = table_3[['app_name','recommend']][table_3['year_posted']==year].groupby('app_name').sum().nlargest(3,'recommend').reset_index()


In [31]:
top_3

,app_name,recommend
0,Counter-strike: global offensive,1401
1,Team fortress 2,610
2,Garry's mod,336


In [32]:
# Crea el diccionario con los puestos y los nombres de las aplicaciones
diccionario_resultado = {f'Puesto{i+1}': juego for i, juego in enumerate(top_3['app_name'])}

In [33]:
[diccionario_resultado]

[{'Puesto1': 'Counter-strike: global offensive',
  'Puesto2': 'Team fortress 2',
  'Puesto3': "Garry's mod"}]

## Funcion UsersRecommend

In [9]:
def user_reviews_recommend(year: int):
  path_endpoint_3 = os.path.join('..','data','clear','03_users_recommend.csv.gz')
  table_3 = pd.read_csv(path_endpoint_3)
  
  top_3 = table_3[['app_name','recommend']][table_3['year_posted']==year].groupby('app_name').sum().nlargest(3,'recommend').reset_index()
  
  diccionario_resultado = {f'Puesto{i+1}': juego for i, juego in enumerate(top_3['app_name'])}
  
  return [diccionario_resultado,]
  

In [10]:
user_reviews_recommend(2015)

,app_name,recommend
0,Counter-strike: global offensive,1401
1,Team fortress 2,610
2,Garry's mod,336
